In [63]:
import pandas as pd
import numpy as np
import re
import json

In [2]:
import spacy
nlp = spacy.load("en_core_web_md")

In [5]:
master = pd.read_pickle("old_named_entities.pkl")

In [46]:
spacy_entities = ["PERSON", "ORG", "GPE", "LOC"]

In [41]:
RE_EMAIL = re.compile(
    r"(?:[a-z0-9!#$%&'*+/=?^_`{|}~-]+(?:\.[a-z0-9!#$%&'*+/=?^_`{|}~-]+)*|(?:[\x01-\x08\x0b\x0c\x0e-\x1f\x21\x23-\x5b\x5d-\x7f]|\\[\x01-\x09\x0b\x0c\x0e-\x7f])*)@(?:(?:[a-z0-9](?:[a-z0-9-]*[a-z0-9])?\.)+[a-z0-9](?:[a-z0-9-]*[a-z0-9])?|\[(?:(?:25[0-5]|2[0-4][0-9]|[01]?[0-9][0-9]?)\.){3}(?:25[0-5]|2[0-4][0-9]|[01]?[0-9][0-9]?|[a-z0-9-]*[a-z0-9]:(?:[\x01-\x08\x0b\x0c\x0e-\x1f\x21-\x5a\x53-\x7f]|\\[\x01-\x09\x0b\x0c\x0e-\x7f])+)\])")
RE_URL = re.compile(r"((https?|ftp|smtp):\/\/)?(www.)?[a-z0-9]+\.[a-z]+(\/[a-zA-Z0-9#]+\/?)*", re.IGNORECASE)
RE_RELAX_PHONE = re.compile(r'(\(? ?[\d]{2,3} ?\)?.{,3}?){2,}')


In [42]:
entities_map = {
    "email": RE_EMAIL,
    "url": RE_URL,
    "phone": RE_RELAX_PHONE
}

In [53]:
non_empty = master[master.blank == 0]

In [61]:
docccano_docs = []
for filename, group in master.groupby("filename"):
    print(filename)
    text = "\n".join(group.line)
    entities = []
    
    doc = nlp(text)
    for ent, pat in entities_map.items():
        match = pat.search(text, re.IGNORECASE)
        if match:
            start, end = match.span()
            entities.append([start, end, ent])

    relevant = [ent for ent in doc.ents if ent.label_ in spacy_entities]
    for ent in relevant:
        entities.append([ent.start_char, ent.end_char, ent.label_])
    
    docccano_docs.append({
        "text": text,
        "labels": entities
    })

0.txt
1.txt
10.txt
100.txt
101.txt
102.txt
103.txt
104.txt
105.txt
107.txt
108.txt
109.txt
11.txt
110.txt
111.txt
112.txt
113.txt
114.txt
115.txt
116.txt
117.txt
118.txt
119.txt
12.txt
120.txt
121.txt
122.txt
123.txt
124.txt
125.txt
126.txt
127.txt
128.txt
129.txt
13.txt
130.txt
131.txt
132.txt
133.txt
134.txt
135.txt
136.txt
137.txt
138.txt
139.txt
14.txt
140.txt
141.txt
142.txt
143.txt
144.txt
145.txt
146.txt
147.txt
148.txt
149.txt
15.txt
150.txt
151.txt
152.txt
153.txt
155.txt
156.txt
157.txt
158.txt
159.txt
16.txt
160.txt
161.txt
162.txt
163.txt
164.txt
165.txt
166.txt
167.txt
168.txt
169.txt
17.txt
170.txt
171.txt
172.txt
173.txt
174.txt
175.txt
176.txt
177.txt
178.txt
179.txt
18.txt
180.txt
181.txt
182.txt
183.txt
184.txt
185.txt
186.txt
187.txt
188.txt
189.txt
19.txt
190.txt
191.txt
192.txt
193.txt
194.txt
195.txt
196.txt
197.txt
198.txt
199.txt
2.txt
20.txt
200.txt
201.txt
202.txt
203.txt
204.txt
205.txt
206.txt
207.txt
208.txt
209.txt
21.txt
210.txt
211.txt
212.txt
213.txt
21

In [64]:
jsonl = "\n".join([json.dumps(x, ensure_ascii=False) for x in docccano_docs])

In [67]:
with open("doccano_entities.jsonl", "w+", encoding="utf-8") as f:
    f.write(jsonl)

In [70]:
len(master.line)

4188

In [71]:
len(master.line.unique())

2359